In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Export Graph

In [1]:
!git clone https://github.com/Unity-Technologies/ml-agents.git
!pip3 install mlagents

Cloning into 'ml-agents'...
remote: Enumerating objects: 388, done.
remote: Counting objects: 100% (388/388), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 17561 (delta 206), reused 266 (delta 138), pack-reused 17173
Receiving objects: 100% (17561/17561), 392.82 MiB | 36.40 MiB/s, done.
Resolving deltas: 100% (11519/11519), done.
     |████████████████████████████████| 122kB 2.0MB/s 
     |████████████████████████████████| 1.1MB 6.2MB/s 
     |████████████████████████████████| 51kB 18.0MB/s 
     |████████████████████████████████| 48.1MB 1.4MB/s 
     |████████████████████████████████| 8.8MB 33.0MB/s 
     |████████████████████████████████| 12.2MB 26.2MB/s 
     |████████████████████████████████| 3.1MB 32.1MB/s 
     |████████████████████████████████| 890kB 37.5MB/s 
     |████████████████████████████████| 337kB 40.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR:

In [0]:
!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/Models/Good/2019_5_7_19_42_14/frozen_inference_graph.pb frozen_graph.pb

In [10]:
!cd ml-agents/ml-agents/mlagents/trainers; python tensorflow_to_barracuda.py /content/frozen_graph /content/holoselecta.nn

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
Converting /content/frozen_graph.pb to /content/holoselecta.nn
IGNORED: Cast unknown layer
IGNORED: Shape unknown layer
IGNORED: Shape unknown layer
IGNORED: TensorArrayV3 unknown layer
IGNORED: Enter unknown layer
IGNORED: TensorArrayV3 unknown layer
IGNORED: Enter unknown layer
IGNORED: Merge unknown layer
IGNORED: Enter unknown layer
IGNORED: TensorArrayV3 unknown layer
IGNORED: Enter unknown layer
IGNORED: Merge unknown layer
IGNORED: Enter unknown layer
IGNORED: Enter unknown layer
IGNORED: Range unknown layer
IGNORED: TensorArrayScatterV3 unknown layer
IGNORED: Enter unknown layer
IGNORED: Enter unknown layer
IGNORED: Merge unknown layer
IGNORED: Less unknown layer
IGNORED: LoopCond unknown layer
IGNORED: Switch 

# Export Images

In [0]:
import os
import numpy as np
import cv2


def resize_padded_with_bndboxes(width_old, height_old, filepath_old, bndboxs_old, length):
  """resizes the image ot a squared padded image. Where the longest
  side will match their target length. The other dimension is padded
  equally on both sides with zeros until it as well matches length.
  Args:
    width_old: the old width
    height_old: the old height
    filepath_old: the old filepath
    bndboxs: a list of bounding boxes
    length: target length
  Returns:
    width: the new width
    length: the new length
    filepath: the new filepath
    bndboxs: the new boundingboxes
  """
  #First resize the image
  width_old, height_old, filepath, bndboxs_old = resize_scaled_with_bndboxes(width_old,
                                                                 height_old,
                                                                 filepath_old,
                                                                 bndboxs_old,
                                                                 length)
  
  #necessary padding, handle uneven numbers
  pad_x = 0
  pad_y = 0
  if height_old < length:
    pad_size = length - height_old
    pad_x = int(pad_size/2)
  if width_old < length:
    pad_size = length - width_old
    pad_y = int(pad_size/2)
  
  #create empty image array and embedd the image in it
  image = np.zeros([length, length, 3])
  image_old = cv2.imread(filepath)
  image[pad_x:(pad_x + height_old), pad_y:(pad_y + width_old), :] = image_old
  cv2.imwrite(filepath, image)
  
  #redo boundingboxes
  bndboxs = []
  for bndbox in bndboxs_old:
    bndboxs.append({})
    bndboxs[-1]["xmin"] = (bndbox["xmin"] * width_old + pad_y) / float(length)
    bndboxs[-1]["xmax"] = (bndbox["xmax"] * width_old + pad_y) / float(length)
    bndboxs[-1]["ymin"] = (bndbox["ymin"] * height_old  + pad_x) / float(length)
    bndboxs[-1]["ymax"] = (bndbox["ymax"] * height_old  + pad_x) / float(length)
    bndboxs[-1]["class_name"] = bndbox["class_name"]
    bndboxs[-1]["class"] = bndbox["class"]
    
    assert all([(float(val) >= 0.0 and float(val) <= 1.0) for key, val in bndboxs[-1].items() if key in ["xmin", "xmax", "ymin", "ymax"]])
  
  return length, length, filepath, bndboxs

In [0]:
import os
import cv2

def resize_scaled_with_bndboxes(width_old, height_old, filepath_old, bndboxs_old, max_length):
  """resize the image and the bounding boxes accordingly
  It checks if a resized image already exists and otherwise
  creates one. the boundingbox coordinates are normalized
  for calculated since they need to be normalized later on
  anyway.
  Args:
    width_old: the old width
    height_old: the old height
    filepath_old: the old filepath
    bndboxs: a list of bounding boxes
    max_length = the size in pixel of the longest side
  Returns:
    width: the new width
    length: the new length
    filepath: the new filepath
    bndboxs: the new boundingboxes
    
  """
  #find a scaler from the longest side and resize the image
  if width_old > height_old:
    scaler = width_old / max_length
  else:
    scaler = height_old / max_length
  
  height = int(height_old / scaler) 
  width = int(width_old / scaler)
  cv_img = cv2.imread(filepath_old, cv2.IMREAD_COLOR)
  cv_img_resized = cv2.resize(cv_img, (width,height))
  filename = filepath_old.split(os.sep)[-1]
  filename = ".".join(filename.split(".")[:-1]) + "_resized.{}".format(filename.split(".")[-1])
  if not os.path.exists("tmp"):
    os.mkdir("tmp")
  filepath = os.path.join("tmp", filename)
  cv2.imwrite(filepath, cv_img_resized)
  
  bndboxs = []
  for bndbox in bndboxs_old:
    bndboxs.append({})
    bndboxs[-1]["xmin"] = bndbox["xmin"] / float(width_old)
    bndboxs[-1]["xmax"] = bndbox["xmax"] / float(width_old)
    bndboxs[-1]["ymin"] = bndbox["ymin"] / float(height_old)
    bndboxs[-1]["ymax"] = bndbox["ymax"] / float(height_old)
    bndboxs[-1]["class_name"] = bndbox["class_name"]
    bndboxs[-1]["class"] = bndbox["class"]
    
    assert all([(float(val) >= 0.0 and float(val) <= 1.0) for key, val in bndboxs[-1].items() if key in ["xmin", "xmax", "ymin", "ymax"]])
  
  return width, height, filepath, bndboxs

In [0]:
width, height, filepath, bndboxs= resize_padded_with_bndboxes(3456,
                            4608,
                            "gdrive/My Drive/Masterarbeit/ObjectDetection/Images/_Automat/IMG_20190502_180150.jpg",
                            [],
                            6430)
print(filepath)

tmp/IMG_20190502_180150_resized.jpg


In [0]:
!mv tmp/IMG_20190502_180150_resized.jpg /content/IMG_20190502_180150_resized.jpg